# EDA: Exploratory Data Analysis

## Problem Tanımı

Bu projede, **oyun sektöründe kullanıcı segmentasyonu** problemi üzerinde çalışıyoruz. Amacımız, oyun kullanıcılarını davranışsal ve demografik özelliklerine göre anlamlı segmentlere ayırmak ve her segment için özelleştirilmiş stratejiler geliştirmektir.

### Business Impact
- **Kişiselleştirilmiş Pazarlama**: Her segment için özel kampanyalar ve içerikler
- **Kullanıcı Tutma**: Segment bazlı retention stratejileri ile churn oranını azaltma
- **Gelir Optimizasyonu**: Yüksek değerli kullanıcıları belirleyip özel teklifler sunma
- **Ürün Geliştirme**: Segment ihtiyaçlarına göre oyun özelliklerini optimize etme
- **Kaynak Yönetimi**: Marketing ve customer success kaynaklarını verimli kullanma

### Segmentasyon Yaklaşımı
- **Unsupervised Learning**: K-Means, DBSCAN, Hierarchical Clustering
- **Feature Engineering**: Kullanıcı davranış metrikleri ve engagement skorları
- **Segment Profilleme**: Her segmentin özelliklerini analiz edip isimlendirme


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import sys

# Add src to path
sys.path.append(str(Path('..').resolve()))
from src.config import *
from src.data_loader import load_gaming_dataset, create_sample_gaming_dataset

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")


## Veri Yükleme


In [ ]:
# Load dataset
try:
    # Try to load existing dataset
    df = load_gaming_dataset(RAW_DATA_DIR)
    if df is None or len(df) == 0:
        raise FileNotFoundError("Dataset not found")
    print("✅ Dataset loaded successfully!")
except FileNotFoundError:
    print("Dataset not found. Creating sample dataset...")
    df = create_sample_gaming_dataset(n_samples=20000, save_path=TRAIN_FILE)
    print("✅ Sample dataset created!")

print(f"\nDataset shape: {df.shape}")
print(f"Rows: {df.shape[0]:,}")
print(f"Columns: {df.shape[1]}")
print(f"\nFirst few rows:")
df.head()


## Temel Bilgiler ve Veri Yapısı


In [ ]:
# Dataset info
print("=" * 60)
print("DATASET INFORMATION")
print("=" * 60)
df.info()

print("\n" + "=" * 60)
print("MISSING VALUES")
print("=" * 60)
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing Percentage': missing_pct
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
if len(missing_df) > 0:
    print(missing_df)
else:
    print("✅ No missing values found!")

print("\n" + "=" * 60)
print("DUPLICATE ROWS")
print("=" * 60)
duplicates = df.duplicated().sum()
print(f"Duplicate rows: {duplicates} ({duplicates/len(df)*100:.2f}%)")


## Değişken Açıklamaları

### Demografik Özellikler
- **user_id**: Kullanıcı benzersiz kimliği
- **age**: Kullanıcı yaşı
- **gender**: Cinsiyet (Male, Female, Other)
- **country**: Ülke
- **device_type**: Cihaz tipi (Mobile, PC, Console)

### Oyun Davranışı
- **total_sessions**: Toplam oyun oturumu sayısı
- **total_playtime_hours**: Toplam oyun süresi (saat)
- **avg_session_duration_minutes**: Ortalama oturum süresi (dakika)
- **max_level_reached**: Ulaşılan maksimum seviye
- **levels_completed**: Tamamlanan seviye sayısı
- **quests_completed**: Tamamlanan görev sayısı
- **achievements_unlocked**: Açılan başarı sayısı

### Engagement Metrikleri
- **days_since_last_login**: Son girişten bu yana geçen gün
- **days_since_registration**: Kayıt olunduğundan bu yana geçen gün
- **login_frequency_per_week**: Haftalık giriş sıklığı
- **friend_count**: Arkadaş sayısı
- **guild_member**: Guild üyeliği (0/1)

### Satın Alma Davranışı
- **total_spent_usd**: Toplam harcama (USD)
- **purchase_count**: Satın alma sayısı
- **avg_purchase_value**: Ortalama satın alma değeri
- **last_purchase_days_ago**: Son satın alımdan bu yana geçen gün
- **premium_subscription**: Premium abonelik durumu (0/1)

### Performans Metrikleri
- **win_rate**: Kazanma oranı
- **avg_score**: Ortalama skor
- **pvp_matches_played**: PvP maç sayısı
- **pve_missions_completed**: PvE görev sayısı

### Sosyal Özellikler
- **chat_messages_sent**: Gönderilen chat mesajı sayısı
- **reviews_written**: Yazılan yorum sayısı
- **events_participated**: Katılım sağlanan etkinlik sayısı

### Türetilmiş Özellikler
- **playtime_per_session**: Oturum başına oyun süresi
- **spending_per_session**: Oturum başına harcama
- **level_completion_rate**: Seviye tamamlama oranı
- **engagement_score**: Genel engagement skoru


## Sayısal Değişkenlerin İstatistiksel Özeti


In [ ]:
# Statistical summary for numerical features
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# Remove user_id if it's numeric but not meaningful
if 'user_id' in numerical_cols:
    numerical_cols.remove('user_id')

print("=" * 60)
print("NUMERICAL FEATURES SUMMARY")
print("=" * 60)
df[numerical_cols].describe().T


## Kategorik Değişkenlerin Dağılımı


In [ ]:
# Categorical features distribution
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
if 'user_id' in categorical_cols:
    categorical_cols.remove('user_id')
if 'registration_date' in categorical_cols:
    categorical_cols.remove('registration_date')

print("=" * 60)
print("CATEGORICAL FEATURES DISTRIBUTION")
print("=" * 60)

for col in categorical_cols:
    print(f"\n{col}:")
    print(df[col].value_counts())
    print("-" * 40)


## Görselleştirmeler

### 1. Demografik Dağılımlar


In [ ]:
# Demographic distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Age distribution
axes[0, 0].hist(df['age'], bins=30, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Age Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Age')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(df['age'].mean(), color='red', linestyle='--', label=f'Mean: {df["age"].mean():.1f}')
axes[0, 0].legend()

# Gender distribution
gender_counts = df['gender'].value_counts()
axes[0, 1].bar(gender_counts.index, gender_counts.values, color=['#3498db', '#e74c3c', '#9b59b6'])
axes[0, 1].set_title('Gender Distribution', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Gender')
axes[0, 1].set_ylabel('Count')
for i, v in enumerate(gender_counts.values):
    axes[0, 1].text(i, v, str(v), ha='center', va='bottom')

# Country distribution (top 10)
country_counts = df['country'].value_counts().head(10)
axes[1, 0].barh(country_counts.index, country_counts.values, color='#2ecc71')
axes[1, 0].set_title('Top 10 Countries', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('User Count')

# Device type distribution
device_counts = df['device_type'].value_counts()
axes[1, 1].pie(device_counts.values, labels=device_counts.index, autopct='%1.1f%%', startangle=90)
axes[1, 1].set_title('Device Type Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()


### 2. Oyun Davranışı Dağılımları


In [ ]:
# Gaming behavior distributions
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Total sessions
axes[0, 0].hist(df['total_sessions'], bins=50, edgecolor='black', alpha=0.7, color='#3498db')
axes[0, 0].set_title('Total Sessions Distribution', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Total Sessions')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(df['total_sessions'].median(), color='red', linestyle='--', 
                    label=f'Median: {df["total_sessions"].median():.0f}')
axes[0, 0].legend()

# Total playtime
axes[0, 1].hist(df['total_playtime_hours'], bins=50, edgecolor='black', alpha=0.7, color='#e74c3c')
axes[0, 1].set_title('Total Playtime (Hours) Distribution', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Total Playtime (Hours)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].axvline(df['total_playtime_hours'].median(), color='red', linestyle='--',
                    label=f'Median: {df["total_playtime_hours"].median():.1f}h')
axes[0, 1].legend()

# Max level reached
axes[0, 2].hist(df['max_level_reached'], bins=30, edgecolor='black', alpha=0.7, color='#2ecc71')
axes[0, 2].set_title('Max Level Reached Distribution', fontsize=12, fontweight='bold')
axes[0, 2].set_xlabel('Max Level')
axes[0, 2].set_ylabel('Frequency')

# Total spent
axes[1, 0].hist(df[df['total_spent_usd'] > 0]['total_spent_usd'], bins=50, 
                edgecolor='black', alpha=0.7, color='#f39c12')
axes[1, 0].set_title('Total Spent (USD) - Paying Users Only', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Total Spent (USD)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_xlim(0, df[df['total_spent_usd'] > 0]['total_spent_usd'].quantile(0.95))

# Engagement score
axes[1, 1].hist(df['engagement_score'], bins=50, edgecolor='black', alpha=0.7, color='#9b59b6')
axes[1, 1].set_title('Engagement Score Distribution', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Engagement Score')
axes[1, 1].set_ylabel('Frequency')

# Login frequency
axes[1, 2].hist(df['login_frequency_per_week'], bins=50, edgecolor='black', alpha=0.7, color='#1abc9c')
axes[1, 2].set_title('Login Frequency per Week', fontsize=12, fontweight='bold')
axes[1, 2].set_xlabel('Logins per Week')
axes[1, 2].set_ylabel('Frequency')
axes[1, 2].set_xlim(0, df['login_frequency_per_week'].quantile(0.95))

plt.tight_layout()
plt.show()

# Summary statistics
print("=" * 60)
print("GAMING BEHAVIOR SUMMARY")
print("=" * 60)
print(f"Average sessions per user: {df['total_sessions'].mean():.1f}")
print(f"Average playtime per user: {df['total_playtime_hours'].mean():.1f} hours")
print(f"Average max level: {df['max_level_reached'].mean():.1f}")
print(f"Paying users: {(df['total_spent_usd'] > 0).sum()} ({(df['total_spent_usd'] > 0).mean()*100:.1f}%)")
print(f"Average spending (paying users): ${df[df['total_spent_usd'] > 0]['total_spent_usd'].mean():.2f}")
print(f"Premium subscribers: {df['premium_subscription'].sum()} ({df['premium_subscription'].mean()*100:.1f}%)")


In [ ]:
# Correlation analysis for key numerical features
key_features = [
    'total_sessions', 'total_playtime_hours', 'max_level_reached',
    'total_spent_usd', 'login_frequency_per_week', 'engagement_score',
    'win_rate', 'avg_score', 'purchase_count'
]

corr_matrix = df[key_features].corr()

plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix - Key Features', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Top correlations
print("=" * 60)
print("TOP POSITIVE CORRELATIONS")
print("=" * 60)
corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_pairs.append((
            corr_matrix.columns[i],
            corr_matrix.columns[j],
            corr_matrix.iloc[i, j]
        ))
corr_df = pd.DataFrame(corr_pairs, columns=['Feature 1', 'Feature 2', 'Correlation'])
corr_df = corr_df.sort_values('Correlation', ascending=False)
print(corr_df.head(10).to_string(index=False))


### 4. İlişki Analizleri (Scatter Plots)


In [ ]:
# Key relationships for segmentation
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Playtime vs Spending
axes[0, 0].scatter(df['total_playtime_hours'], df['total_spent_usd'], 
                   alpha=0.5, s=20, c=df['engagement_score'], cmap='viridis')
axes[0, 0].set_xlabel('Total Playtime (Hours)', fontsize=11)
axes[0, 0].set_ylabel('Total Spent (USD)', fontsize=11)
axes[0, 0].set_title('Playtime vs Spending (colored by Engagement)', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
cbar = plt.colorbar(axes[0, 0].collections[0], ax=axes[0, 0])
cbar.set_label('Engagement Score', fontsize=10)

# Sessions vs Level
axes[0, 1].scatter(df['total_sessions'], df['max_level_reached'], 
                   alpha=0.5, s=20, c=df['win_rate'], cmap='plasma')
axes[0, 1].set_xlabel('Total Sessions', fontsize=11)
axes[0, 1].set_ylabel('Max Level Reached', fontsize=11)
axes[0, 1].set_title('Sessions vs Level (colored by Win Rate)', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
cbar = plt.colorbar(axes[0, 1].collections[0], ax=axes[0, 1])
cbar.set_label('Win Rate', fontsize=10)

# Login Frequency vs Engagement
axes[1, 0].scatter(df['login_frequency_per_week'], df['engagement_score'], 
                   alpha=0.5, s=20, c=df['total_spent_usd'], cmap='YlOrRd')
axes[1, 0].set_xlabel('Login Frequency per Week', fontsize=11)
axes[1, 0].set_ylabel('Engagement Score', fontsize=11)
axes[1, 0].set_title('Login Frequency vs Engagement (colored by Spending)', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
cbar = plt.colorbar(axes[1, 0].collections[0], ax=axes[1, 0])
cbar.set_label('Total Spent (USD)', fontsize=10)

# Age vs Spending
axes[1, 1].scatter(df['age'], df['total_spent_usd'], 
                   alpha=0.5, s=20, c=df['total_playtime_hours'], cmap='coolwarm')
axes[1, 1].set_xlabel('Age', fontsize=11)
axes[1, 1].set_ylabel('Total Spent (USD)', fontsize=11)
axes[1, 1].set_title('Age vs Spending (colored by Playtime)', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
cbar = plt.colorbar(axes[1, 1].collections[0], ax=axes[1, 1])
cbar.set_label('Total Playtime (Hours)', fontsize=10)

plt.tight_layout()
plt.show()


## EDA Bulguları ve Özet

### Önemli Bulgular

1. **Demografik Dağılım**
   - Kullanıcıların yaş dağılımı normal dağılıma yakın
   - Cinsiyet dağılımı dengeli
   - Mobil cihazlar en yaygın platform

2. **Oyun Davranışı**
   - Kullanıcılar arasında önemli davranış farklılıkları var
   - Playtime ve spending arasında pozitif korelasyon
   - Engagement skoru segmentasyon için önemli bir metrik

3. **Segmentasyon İçin Önemli Özellikler**
   - **Engagement Score**: Genel kullanıcı aktivitesi
   - **Total Spent**: Monetization potansiyeli
   - **Total Playtime**: Oyun bağlılığı
   - **Login Frequency**: Aktiflik seviyesi
   - **Max Level**: İlerleme durumu

4. **Korelasyonlar**
   - Yüksek korelasyonlu özellikler segmentasyon için dikkatle seçilmeli
   - Multicollinearity problemi olabilir

### Segmentasyon Stratejisi

Clustering için önerilen özellikler:
- Engagement metrikleri (sessions, playtime, login frequency)
- Monetization metrikleri (spending, purchase count)
- Progression metrikleri (level, achievements)
- Social metrikleri (friends, guild membership)

### Sonraki Adımlar

1. **Baseline Model**: En basit feature set ile K-Means clustering
2. **Feature Engineering**: Yeni özellikler türetme ve seçme
3. **Model Optimization**: Farklı clustering algoritmaları ve hiperparametreler
4. **Model Evaluation**: Silhouette score, davies-bouldin index gibi metrikler
